In [75]:
import networkx as nx
import re

In [76]:
G = nx.Graph()
with open('16.txt') as f:
    data = f.read().splitlines()
for line in data:
    line = line.replace('valves', 'valve')
    valve_name = line[6:8]
    left, right = line.split('; ')
    rest, flow_rate = left.split('=')
    flow_rate = int(flow_rate)
    destinations = right.split('valve ')
    print(line)
    if ',' in destinations[1]:
        destinations = destinations[1].split(', ')
    else:
        destinations = [destinations[1]]
    print(valve_name, flow_rate, destinations)
    
    # NX
    G.add_node(valve_name, flow_rate=flow_rate, open=False)
    for dest in destinations:
        G.add_edge(valve_name, dest, weight=1)
    

Valve HM has flow rate=0; tunnels lead to valve LS, YS
HM 0 ['LS', 'YS']
Valve IY has flow rate=15; tunnels lead to valve YI, MU, KN, QS, QM
IY 15 ['YI', 'MU', 'KN', 'QS', 'QM']
Valve VI has flow rate=22; tunnels lead to valve LE, SE, RB, JR
VI 22 ['LE', 'SE', 'RB', 'JR']
Valve SE has flow rate=0; tunnels lead to valve VI, AZ
SE 0 ['VI', 'AZ']
Valve QU has flow rate=0; tunnels lead to valve YC, QK
QU 0 ['YC', 'QK']
Valve RB has flow rate=0; tunnels lead to valve AN, VI
RB 0 ['AN', 'VI']
Valve PU has flow rate=0; tunnels lead to valve JR, IM
PU 0 ['JR', 'IM']
Valve OA has flow rate=0; tunnels lead to valve KZ, FR
OA 0 ['KZ', 'FR']
Valve AQ has flow rate=23; tunnels lead to valve FA, QM, GE
AQ 23 ['FA', 'QM', 'GE']
Valve QS has flow rate=0; tunnels lead to valve IM, IY
QS 0 ['IM', 'IY']
Valve HC has flow rate=24; tunnel leads to valve XH
HC 24 ['XH']
Valve QI has flow rate=0; tunnels lead to valve KQ, LS
QI 0 ['KQ', 'LS']
Valve FA has flow rate=0; tunnels lead to valve HA, AQ
FA 0 ['HA',

In [77]:
MINUTES = 30
max_total_flow = 0
START = 'AA'
already_open_or_no_flow = set([x for x in G.nodes if G.nodes[x]['flow_rate'] == 0])
current_node = START
##
flowed = 0
open_flow = 0
passed_minutes = 0
def timestep(passed_minutes, flowed, open_flow):
    passed_minutes += 1
    flowed = flowed + open_flow
    print(f'Flowing {open_flow} at {passed_minutes} minutes. Total flow: {flowed}')
    return passed_minutes, flowed

while True:
    max_potential_flow = 0
    next_node = current_node
    for target in G.nodes:
        path = nx.shortest_path(G, current_node, target)
        potential_flow = (MINUTES - passed_minutes - (len(path) - 1)) * G.nodes[target]['flow_rate']
        
        if potential_flow > max_potential_flow and not target in already_open_or_no_flow:
            max_potential_flow = potential_flow
            next_node = path[1] if len(path) > 1 else target
            print(f'Potential flow of {target} is {potential_flow}. Passed minutes: {passed_minutes} (next node: {next_node})')
            
    if next_node == current_node and not current_node in already_open_or_no_flow:
        # Opening
        passed_minutes, flowed = timestep(passed_minutes, flowed, open_flow)
        already_open_or_no_flow.add(current_node)
        open_flow += G.nodes[current_node]['flow_rate']
        print(f'Opened {current_node} at {passed_minutes} minutes, releasing {G.nodes[current_node]["flow_rate"]} Current flow: {open_flow}')
       

    else:
        # Moving
        passed_minutes, flowed = timestep(passed_minutes, flowed, open_flow) 
        current_node = next_node
        pass

    if passed_minutes == MINUTES:
        break

Potential flow of LS is 81. Passed minutes: 0 (next node: YS)
Potential flow of IY is 315. Passed minutes: 0 (next node: XT)
Potential flow of VI is 462. Passed minutes: 0 (next node: XT)
Potential flow of HC is 576. Passed minutes: 0 (next node: GY)
Potential flow of AI is 600. Passed minutes: 0 (next node: GY)
Flowing 0 at 1 minutes. Total flow: 0
Potential flow of LS is 75. Passed minutes: 1 (next node: AA)
Potential flow of IY is 285. Passed minutes: 1 (next node: AA)
Potential flow of VI is 418. Passed minutes: 1 (next node: AA)
Potential flow of AZ is 420. Passed minutes: 1 (next node: AA)
Potential flow of HC is 576. Passed minutes: 1 (next node: QK)
Potential flow of AI is 600. Passed minutes: 1 (next node: QK)
Flowing 0 at 2 minutes. Total flow: 0
Potential flow of LS is 75. Passed minutes: 2 (next node: RM)
Potential flow of IY is 285. Passed minutes: 2 (next node: XX)
Potential flow of VI is 418. Passed minutes: 2 (next node: XX)
Potential flow of AZ is 420. Passed minutes: 

In [78]:
flowed

1290

In [79]:
all_non_zero_flows = [x for x in G.nodes if G.nodes[x]['flow_rate'] > 0]

In [80]:
len(all_non_zero_flows)

15

In [81]:
class PressureFlow:
    def __init__(self, verbose=False):
        self.flow_rate = 0
        self.released_pressure = 0
        self.passed_minutes = 0
    def increase_flow(self, amount):
        self.timestep()
        self.flow_rate += amount
    def timestep(self):
        self.released_pressure += self.flow_rate
        self.passed_minutes += 1
    def copy(self):
        pf = PressureFlow()
        pf.flow_rate = self.flow_rate
        pf.released_pressure = self.released_pressure
        pf.passed_minutes = self.passed_minutes
        return pf

In [82]:
G = nx.Graph()
with open('16.txt') as f:
    data = f.read().splitlines()
    
for line in data:
    line = line.replace('valves', 'valve')
    valve_name = line[6:8]
    left, right = line.split('; ')
    rest, flow_rate = left.split('=')
    flow_rate = int(flow_rate)
    destinations = right.split('valve ')
    print(line)
    if ',' in destinations[1]:
        destinations = destinations[1].split(', ')
    else:
        destinations = [destinations[1]]
    print(valve_name, flow_rate, destinations)
    
    # NX
    G.add_node(valve_name, flow_rate=flow_rate, open=False)
    for dest in destinations:
        G.add_edge(valve_name, dest, weight=1)
    

Valve HM has flow rate=0; tunnels lead to valve LS, YS
HM 0 ['LS', 'YS']
Valve IY has flow rate=15; tunnels lead to valve YI, MU, KN, QS, QM
IY 15 ['YI', 'MU', 'KN', 'QS', 'QM']
Valve VI has flow rate=22; tunnels lead to valve LE, SE, RB, JR
VI 22 ['LE', 'SE', 'RB', 'JR']
Valve SE has flow rate=0; tunnels lead to valve VI, AZ
SE 0 ['VI', 'AZ']
Valve QU has flow rate=0; tunnels lead to valve YC, QK
QU 0 ['YC', 'QK']
Valve RB has flow rate=0; tunnels lead to valve AN, VI
RB 0 ['AN', 'VI']
Valve PU has flow rate=0; tunnels lead to valve JR, IM
PU 0 ['JR', 'IM']
Valve OA has flow rate=0; tunnels lead to valve KZ, FR
OA 0 ['KZ', 'FR']
Valve AQ has flow rate=23; tunnels lead to valve FA, QM, GE
AQ 23 ['FA', 'QM', 'GE']
Valve QS has flow rate=0; tunnels lead to valve IM, IY
QS 0 ['IM', 'IY']
Valve HC has flow rate=24; tunnel leads to valve XH
HC 24 ['XH']
Valve QI has flow rate=0; tunnels lead to valve KQ, LS
QI 0 ['KQ', 'LS']
Valve FA has flow rate=0; tunnels lead to valve HA, AQ
FA 0 ['HA',

In [83]:
all_non_zero_flows

['LS',
 'IY',
 'VI',
 'AZ',
 'YC',
 'QK',
 'IM',
 'KZ',
 'AQ',
 'HC',
 'KQ',
 'HA',
 'ZJ',
 'RF',
 'AI']

In [84]:
from functools import lru_cache
import itertools

@lru_cache
def get_shortest_path(G, start, end):
    return nx.shortest_path(G, start, end)

In [103]:
max_minutes = 30

from tqdm import tqdm
from math import factorial
import copy


def move_and_maybe_open(G, flow: PressureFlow, current_node, node):
    shortest_path = get_shortest_path(G, current_node, node)
    for _ in range(len(shortest_path)-1):
        flow.timestep()
        if flow.passed_minutes >= max_minutes:
            return G, flow
    if not flow.passed_minutes >= max_minutes:
        open_node(G, flow, node)
    return G, flow

def open_node(G, flow: PressureFlow, node):
    # Open node if smart
    if (G.nodes[node]['flow_rate'] > 0) and not G.nodes[node]['open']:
        flow.increase_flow(G.nodes[node]["flow_rate"])
        G.nodes[node]['open'] = True
        
def calculate_potential(G, flow: PressureFlow, current_node, node):
    # Calculate potential
    if G.nodes[node]['open']:
        return 0
    shortest_path = get_shortest_path(G, current_node, node)
    potential_flow = (max_minutes - flow.passed_minutes - (len(shortest_path) - 1)) * G.nodes[node]['flow_rate']
    return potential_flow

def get_best_opening_combination(G, flow, starting_node):
    possible_nodes = [x for x in G.nodes if calculate_potential(G, flow, starting_node, x) > 0]
    print(possible_nodes)
    if len(possible_nodes) == 0 and flow.passed_minutes < max_minutes:
        while flow.passed_minutes < max_minutes:
            flow.timestep()
        return flow.released_pressure
    elif flow.passed_minutes == max_minutes:
        return flow.released_pressure
    else:
        possible_next_nodes = []
        for node in possible_nodes:
            G_, flow_ = move_and_maybe_open(G.copy(), flow.copy(), starting_node, node)
            possible_next_nodes.append(get_best_opening_combination(G_, flow_, node))
        return max(possible_next_nodes)


In [104]:
flow = PressureFlow()

get_best_opening_combination(G.copy(), flow, 'AA')

['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']
['LS', 'IY', 'VI', 'AZ', 'YC', 'QK', 'IM', 'KZ', 'AQ', 'HC', 'KQ', 'HA', 'ZJ', 'RF', 'AI']

RecursionError: maximum recursion depth exceeded